<h1 align="center"> IS DATA SCIENCE A RISING CAREER IN 2020? </h1>
<h1 align="center"> PART I: EDUCATION LEVEL </h1>
<img align = "center" src="https://i.postimg.cc/zv50Hrh9/data-science-rising-career.jpg" width=500>


<h3> 
    Hello to all experts on Kaggle. I am a new member and would love to hear and learn from all of you. In this Notebook, I try to create functions that can be reusable to check or extract additional information according to the user's preferences and requirements. We hope to receive your recommendations to make my notebook more complete, your comments will be the motivation for me to continue working on the next notebooks. <h3>
    

<h3> Based on the current database (survey questions), we can divide the survey content into 8 topics related to data science. Before going to discuss each topic, I will try firstly to creat some generic functions.
    <h3>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/kaggle-survey-2020/kaggle_survey_2020_responses.csv')

In [ ]:
df.shape

In [ ]:
pd.set_option('display.max_columns', None)
df.head(2)

In [ ]:
pd.set_option('display.max_rows', None)
df.T.iloc[:,0]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

The database is extracted into: the "questions" and the "data" (which is used in this Notebook)

In [ ]:
question = df.iloc[0,:]
df = df.iloc[1:,:]
df['Q12_Part_1'].head(2)

In [ ]:
df.info()

<h3> The database contain all objects, not numerical data. We use the following function to count the data in each column, base on the column name and the gender (Man or Woman). The coldat0 is used to collect column data and coldat2 is used to separately collect data for Man and Woman.<h3>

In [ ]:
def coldat0(serie):
    X0 = df[[i for i in df.columns if serie in i]]
    X = pd.Series(dtype='int')
    for i in X0.columns:
        X[X0[i].value_counts().index[0]] = X0[i].count()
    return X.sort_values()
def coldat2(serie):
    dman = df[(df['Q2']=='Man').values]
    dwoman = df[(df['Q2']=='Woman').values]  
    man0 = dman[[i for i in df.columns if serie in i]]
    man = pd.Series(dtype='int')
    woman0 = dwoman[[i for i in df.columns if serie in i]]
    woman = pd.Series(dtype='int')
    for i in man0.columns:
        man[man0[i].value_counts().index[0]] = man0[i].count()
    for i in woman0.columns:
        woman[woman0[i].value_counts().index[0]] = -woman0[i].count()    
    return man.sort_values(), woman.sort_values()

<h3> The first graphical type we want to plot is the DISTRIBUTION. The Plot_dix function bellow has been created for such purpose.<h3>

In [ ]:
def Plot_dix(title, serie, order):
    check = ['Q7', 'Q9', 'Q10', 'Q12', 'Q14', 'Q16', 'Q17', 'Q18', 'Q19', 'Q23','Q26_A','Q26_B','Q27_A','Q27_B','Q28_A','Q28_B',
             'Q29_A','Q29_B', 'Q31_A', 'Q31_B','Q33_A','Q33_B', 'Q34_A', 'Q34_B','Q35_A','Q35_B','Q36', 'Q37', 'Q39']
        
    f = plt.figure(figsize=(24,7)) 
    
    ax=f.add_subplot(121)
    if serie in check:
        X = coldat0(serie)
    else:
        X = df[serie].value_counts()[order]
    ax.bar(X.index, X, width=0.4,edgecolor='darkgreen', color='grey',linewidth=0.7)
    ma = round(X.max()+1000,3) * 0.02
    for i in X.index:
        ax.annotate(f"{X[i]}", xy=(i, X[i] + ma), va = 'center', ha='center',fontweight='light', fontsize=12, fontfamily='serif',color='black')
    ax.set_xticklabels(X.index, fontfamily='serif',rotation=90, fontweight='bold', fontsize=14)
    f.text(0.1, 0.95, title, fontsize=16, fontweight='bold', fontfamily='serif')   
    ax.grid(axis='y', linestyle='-', alpha=0.4)

    ax=f.add_subplot(122)    
    if serie in check:
        man, woman = coldat2(serie)
    else:
        dman = df[(df['Q2']=='Man').values]
        dwoman = df[(df['Q2']=='Woman').values]
        man = dman[serie].value_counts()[order]
        woman = -dwoman[serie].value_counts()[order]
    mp = round(man.max()+1000,3) * 0.02
    mn = - round(woman.min()-1000,3) * 0.05
    ax.bar(man.index, man, width=0.4, color='green', alpha=0.8, label='Man')
    ax.bar(woman.index, woman, width=0.4, color='red', alpha=0.8, label='Woman')
    for i in man.index:
        ax.annotate(f"{man[i]}", xy=(i, man[i] + mp),va = 'center', ha='center',fontsize=12,fontfamily='serif',color='black')
    for i in woman.index:
        ax.annotate(f"{-woman[i]}", xy=(i, woman[i] - mn), va = 'center', ha='center',fontsize=12,fontfamily='serif',color='black')    
    
    ax.set_xticklabels(man.index, fontfamily='serif', rotation =90, fontweight='bold', fontsize=14)
    ax.legend()
    f.text(0.55, 0.95, title + '\n of man vs woman', fontsize=16, fontweight='bold', fontfamily='serif')  

In order to have the list of order, it's quite simple by using:

In [ ]:
list(df['Q1'].unique())

<h3> The second graphical type we might need to use is the heatmap. A combination of Pivot_table and heatmap plot has been created. <h3>

In [ ]:
def Plot_heatmap(title, serie1, order1, serie2, order2):
    df['count'] = 1
    fig, ax = plt.subplots(1, 1, figsize=(15, 15))

    sns.heatmap(pd.pivot_table(df, values='count', index=[serie1], columns=[serie2], aggfunc=np.sum).loc[order1, order2],cmap='viridis', square=True, linewidth=3, cbar=False, ax=ax,annot=True, fmt='n')
    
    fig.text(0.4, 1, title, fontweight='bold', fontfamily='serif', fontsize=15)
    ax.set_yticklabels(ax.get_yticklabels(), fontfamily='serif', rotation = 0, fontsize=12)
    ax.set_xticklabels(ax.get_xticklabels(), fontfamily='serif', rotation = 90, fontsize=12)
    plt.tight_layout()
    plt.show()

<h3> A third illustration to show the dependency between two series data was also generated by calling the Plot_muldix function. <h3>

In [ ]:
def Plot_Muldix(title, serie1, serie2, order2):
    didj = pd.pivot_table(df, values='count', index=[serie1], columns=[serie2], aggfunc=np.sum).fillna(0).astype(int).loc[:,order2]
    didj = (didj.T / didj.sum(axis=1))
    
    fig, ax = plt.subplots(len(didj.columns),1,figsize=(13, 25), sharex=True)
    for idx2, idx1 in enumerate(didj.columns):
        ax[idx2].text(-1.6 ,0 ,idx1, fontfamily='serif', fontsize=11,ha="right")
        ax[idx2].bar(didj[idx1].index, didj[idx1], color='green', edgecolor='black', linewidth=0.6, width=0.6)
        ax[idx2].set_yticks([])
    
    fig.text(0.13, 0.90, title, fontsize=17, fontweight='bold', fontfamily='serif') 
    plt.subplots_adjust(hspace=0)
    ax[-1].set_xticklabels(order2, fontfamily='serif', rotation=90)
    plt.show()

<h3> And finally, the distribution plot of two series of data has been also created for a single illustration or for a distinguished (gender) plot.

In [ ]:
# https://stackoverflow.com/questions/56337732/how-to-plot-scatter-pie-chart-using-matplotlib
def drawPieMarker(xs, ys, ratios, sizes, colors, ax):
    markers = []
    previous = 0
    # calculate the points of the pie pieces
    for color, ratio in zip(colors, ratios):
        this = 2 * np.pi * ratio + previous
        x  = [0] + np.cos(np.linspace(previous, this, 30)).tolist() + [0]
        y  = [0] + np.sin(np.linspace(previous, this, 30)).tolist() + [0]
        xy = np.column_stack([x, y])
        previous = this
        markers.append({'marker':xy, 's':np.abs(xy).max()**2*np.array(sizes), 'facecolor':color})
    # scatter each of the pie pieces to create pies
    for marker in markers:
        ax.scatter(xs, ys, **marker, alpha=0.7)

def Pivot_ij(serie1, serie2, order1, order2):
    return pd.pivot_table(df, values='count', index=[serie1], columns=[serie2],aggfunc=np.sum).fillna(0).astype(int).loc[order1, order2].stack()

def Pivot_ijk(serie1, serie2, order1, order2, gender):
    return pd.pivot_table(df[df['Q2']==gender], values='count', index=[serie1], columns=[serie2],aggfunc=np.sum).fillna(0).astype(int).loc[order1, order2].stack()

In [ ]:
def Plot_matrix0(title, serie1, serie2, order1, order2):
    
    fig = plt.figure(figsize=(20, 23), dpi=200)
    gs = fig.add_gridspec(5, 5)

    ax_plot = fig.add_subplot(gs[1:4, 0:4]) 
    data_ij = Pivot_ij(serie1, serie2, order1, order2)   
    for idx1 in order1[::-1]:
        for idx2 in order2:
            dat = data_ij[idx1][idx2]
            ax_plot.scatter(idx2, idx1, s=dat, color='darkblue')
    ax_plot.grid(linewidth=0.2, zorder=0)        
    ax_plot.set_yticklabels(idx1, fontfamily='serif', fontsize=15)
    ax_plot.set_xticklabels(idx2, fontfamily='serif', fontsize=15, rotation=90)
# Pos
    ax_pos = fig.add_subplot(gs[0, :4], sharex=ax_plot) 
    data_j = df[serie2].value_counts()[order2]
    ax_pos.bar(data_j.index, data_j, width=0.35, alpha=0.85, color='darkblue')
    plt.setp(ax_pos.get_xticklabels(), visible=False)
# Exp
    ax_exp = fig.add_subplot(gs[1:4, 4], sharey=ax_plot) 
    data_i = df[serie1].value_counts()[order1]
    ax_exp.barh(data_i.index[::-1], data_i[::-1], height=0.35, alpha=0.85, color='darkblue')
    plt.setp(ax_exp.get_yticklabels(), visible=False)
# Spines
    for s in ['top', 'left', 'right', 'bottom']:
        ax_plot.spines[s].set_visible(False)
        ax_pos.spines[s].set_visible(False)
        ax_exp.spines[s].set_visible(False)
    fig.text(0.8, 0.9, title, fontweight='bold', fontfamily='serif', fontsize=35, ha='right') 
    plt.tight_layout()
    plt.show()

In [ ]:
def Plot_matrix2(title, serie1, serie2, order1, order2):
    
    data_ij = Pivot_ij(serie1, serie2, order1, order2)
    data_ij_man = Pivot_ijk(serie1, serie2, order1, order2, 'Man')
    data_ij_woman = Pivot_ijk(serie1,serie2, order1, order2, 'Woman')

    fig = plt.figure(figsize=(20, 23), dpi=200)
    gs = fig.add_gridspec(5, 5)
    ax_plot = fig.add_subplot(gs[1:4, 0:4])
    for idx1 in order1[::-1]:
        for idx2 in order2:
            man = data_ij_man[idx1][idx2]
            woman = data_ij_woman[idx1][idx2]
            manwoman = data_ij[idx1][idx2]
            drawPieMarker([idx2],[idx1], [man/(man+woman), woman/(man+woman)] ,[manwoman*3], ['darkblue', '#990000'], ax=ax_plot)
    ax_plot.grid(linewidth=0.2, zorder=0)        
    ax_plot.set_yticklabels(idx1, fontfamily='serif', fontsize=15)
    ax_plot.set_xticklabels(idx2, fontfamily='serif', fontsize=15, rotation=90)
    ax_pos = fig.add_subplot(gs[0, :4], sharex=ax_plot) 
    data_j_woman = df[df['Q2']=='Woman'][serie2].value_counts()[order2]
    ax_pos.bar(data_j_woman.index, data_j_woman, width=0.4, alpha=0.7, color='#990000')
    data_j_man = df[df['Q2']=='Man'][serie2].value_counts()[order2]
    ax_pos.bar(data_j_man.index, data_j_man, bottom=data_j_woman , width=0.4, alpha=0.7, color='darkblue')
    plt.setp(ax_pos.get_xticklabels(), visible=False)
# Exp
    ax_exp = fig.add_subplot(gs[1:4, 4], sharey=ax_plot) 
    data_i_woman = df[df['Q2']=='Woman'][serie1].value_counts()[order1]
    ax_exp.barh(data_i_woman.index[::-1], data_i_woman[::-1], height=0.4, alpha=0.7, color='#990000')
    data_i_man = df[df['Q2']=='Man'][serie1].value_counts()[order1]
    ax_exp.barh(data_i_man.index[::-1], data_i_man[::-1], left= data_i_woman[::-1],height=0.4, alpha=0.7, color='darkblue')
    plt.setp(ax_exp.get_yticklabels(), visible=False)
    fig.text(0.7, 0.9, title, fontweight='bold', fontfamily='serif', fontsize=20, ha='right') 
    plt.show()

Input parameter preparation ...

In [ ]:
q1order = ['18-21','22-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54','55-59', '60-69', '70+']
q3order = ['India','United States of America','Brazil','Japan','Russia',
           'United Kingdom of Great Britain and Northern Ireland','Nigeria','China','Germany','Turkey',
           'Spain','France','Canada','Indonesia','Pakistan','Taiwan','Italy','Australia','Mexico', 'South Korea']
q4order = ['I prefer not to answer','No formal education past high school','Professional degree','Some college/university study without earning a bachelor’s degree',
    'Bachelor’s degree','Master’s degree','Doctoral degree']
q5order =['Currently not employed', 'Student','Data Analyst','Business Analyst', 'Data Scientist','Software Engineer',
          'Research Scientist', 'Machine Learning Engineer', 'Product/Project Manager', 'Data Engineer', 'Statistician',
          'DBA/Database Engineer','Other']
q6order = ['I have never written code','< 1 years','1-2 years','3-5 years', '5-10 years', '10-20 years', '20+ years']
q7order = ['Python','R','SQL','C','C++','Java','Javascript','Julia','Swift','Bash','MATLAB','None','Other']
q8order = ['Python','R','C++','SQL','Java','MATLAB','C','Javascript','Julia','Swift','Bash','None','Other']
q9order = ['None','Other']
q10order = ['None','Other']
q11order = ['A personal computer or laptop','A cloud computing platform (AWS, Azure, GCP, hosted notebooks, etc)',
            'A deep learning workstation (NVIDIA GTX, LambdaLabs, etc)','None','Other']
q12order = ['GPUs','TPUs','None']
q13order = ['Never','Once','2-5 times','6-25 times','More than 25 times']
q14order = ['Matplotlib','Seaborn','Plotly / Plotly Express','Ggplot / ggplot2','Shiny','D3 js','Altair','Bokeh','Geoplotlib','Leaflet / Folium','None','Other']
q15order = ['I do not use machine learning methods','Under 1 year','1-2 years','2-3 years','3-4 years','4-5 years',
            '5-10 years','10-20 years','20 or more years']
q16order = ['None']
q17order = ['Linear or Logistic Regression','Decision Trees or Random Forests','Gradient Boosting Machines (xgboost, lightgbm, etc)',
            'Bayesian Approaches','Evolutionary Approaches', 'Dense Neural Networks (MLPs, etc)','Convolutional Neural Networks',
            'Generative Adversarial Networks','Recurrent Neural Networks','Transformer Networks (BERT, gpt-3, etc)','None','Other']
q18order = ['None']
q19order = ['None']
q20order = ['0-49 employees','50-249 employees','250-999 employees','1000-9,999 employees','10,000 or more employees']
q21order = ['0','1-2','3-4','5-9','10-14','15-19','20+']
q22order = ['We use ML methods for generating insights (but do not put working models into production)',
            'We recently started using ML methods (i.e., models in production for less than 2 years)',
            'We have well established ML methods (i.e., models in production for more than 2 years)',
            'We are exploring ML methods (and may one day put a model into production)',
            'No (we do not use ML methods)','I do not know']
q23order = ['Analyze and understand data to influence product or business decisions',
            'Build prototypes to explore applying machine learning to new areas',
            'Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data',
            'Build and/or run a machine learning service that operationally improves my product or workflows',
            'Experimentation and iteration to improve existing ML models',
            'Do research that advances the state of the art of machine learning',
            'None of these activities are an important part of my role at work']
q24order = ['$0-999', '1,000-1,999', '2,000-2,999', '3,000-3,999', '4,000-4,999', '5,000-7,499', '7,500-9,999',
'10,000-14,999','15,000-19,999', '20,000-24,999', '25,000-29,999', '30,000-39,999', '40,000-49,999', '50,000-59,999', '60,000-69,999', '70,000-79,999', '80,000-89,999', '90,000-99,999',
'100,000-124,999', '125,000-149,999',  '150,000-199,999', '200,000-249,999',  '250,000-299,999', '300,000-500,000', '> $500,000']
q25order = ['$0 ($USD)','$1-$99','$100-$999','$1000-$9,999','$10,000-$99,999','$100,000 or more ($USD)']
q26aorder = ['None']
q27aorder = ['None']
q28aorder = ['None']
q29aorder = ['None']
q30order = ['MySQL ','PostgresSQL ','Microsoft SQL Server ','MongoDB ','Oracle Database ','Google Cloud BigQuery ',
           'SQLite ','Amazon Redshift ','Microsoft Azure Data Lake Storage ','Amazon Athena ','Snowflake ',
           'IBM Db2 ','Amazon DynamoDB ','Google Cloud SQL ','Microsoft Access ','Google Cloud Firestore ','Other']
q31aorder = ['None','Other']
q32order = ['Microsoft Power BI','Tableau','Google Data Studio','Other','SAP Analytics Cloud ','TIBCO Spotfire',
            'Qlik','Salesforce','Looker','Amazon QuickSight','Alteryx ','Domo']
q33aorder = ['None','Other']
q34aorder = ['None','Other']
q35aorder = ['None','Other']
q36order = ['None','Other']
q37order = ['None','Other']
q38order = ['Local development environments (RStudio, JupyterLab, etc.)',
            'Basic statistical software (Microsoft Excel, Google Sheets, etc.)',
            'Business intelligence software (Salesforce, Tableau, Spotfire, etc.)',
            'Cloud-based data software & APIs (AWS, GCP, Azure, etc.)',
            'Advanced statistical software (SPSS, SAS, etc.)','Other']
q39order = ['None','Other']

<H3> Let's start <h3>
    
 <h3> Topic 1: General information <h3>

    Q1: What is your age (# years)?
    Q2: What is your gender?
    Q3: In which country do you currently reside?

In [ ]:
Plot_dix('Age distribution','Q1',q1order)

We can see that, in 2020, more than 55% of the total is under 30 years old and nearly 80% are Man.

In [ ]:
Plot_dix('Residence contry distribution','Q3',q3order)

Distribution for the top 20 countries in the number of responses. India is still the "top" contry in number, and then the US. 
The rate of woman is around 20-25%.

**<h3> Topic 2. Education and coding experience <h3>**
    + Q4: What is the highest level of formal education that you have attained or plan to attain within the next 2 years?
    + Q6: For how many years have you been writing code and/or programming?
    + Q37_Part_1: On which platforms have you begun or completed data science courses?

In [ ]:
Plot_dix('Current education level or plan to attain (next 2 years) distribution','Q4',q4order)

Wow, this data is quite supprising, over 75% earned bachelor or master degree. Upto 40% are masters and 12% are PhD holders.

A question can be posed here: is the "career switch" to Data Science from other scientific or technical fields dominate? How many percent of people graduated from computer science in this group?

In [ ]:
Plot_heatmap('Pivot Table : Education & Country', 'Q3', q3order, 'Q4',  q4order)

In almost country, there are the most masters.

In [ ]:
Plot_dix('Writing code and/or programming experience','Q6',q6order)

You see, there are over 70% of people with less than 5 years of working and 47% with more than 2 years. Meanwhile, MANY EMPLOYERS require applicants to have more than 5 years of experience to be recruited.

In [ ]:
Plot_heatmap('Pivot Table : Education & coding experience', 'Q6',  q6order, 'Q4', q4order)

Most of Data Expert are graduated or after-graduated and having 1-5 years of coding experience.

In [ ]:
Plot_matrix2('Gender & Education & Age','Q1','Q4',q1order, q4order)

Most people are very young (<30), bachelor degree holders, women aged 24-30 with master degree are majority, while men, by contrast, maybe women are diligent and braver than men when continuing to master.

The first step in your journey to becoming a data scientist is probably to choose some form of education or courses to enhance your knowledge. But how do you know what kind of education you need or where to find related courses?

In [ ]:
Plot_dix('Platforms to begin or complete data science courses','Q37',q37order)

As observed in 2019, a majority of them, 21%, have used Coursera’s data science courses. unlike last year, Kaggle has surpassed University, Udemy, and Datacamp to become the second choice for beginners. Congratulations to Kaggle on this outstanding achievement.

**<h3> Topic 3. Job position & company<h3>**
    + Q5: Select the title most similar to your current role (or most recent title if retired)?
    + Q20: What is the size of the company where you are employed?
    + Q21: Approximately how many individuals are responsible for data science workloads at your place of business?
    + Q24: What is your current yearly compensation (approximate $USD)?

In [ ]:
Plot_dix('Current job position distribution','Q5',q5order)

After "Student", "Data Scientist" is the most common job role.
The "Unemployed" rate is 8.5%, if including the "Student" rate, this rate is: 35%. Rub.

In [ ]:
Plot_dix('Size of data science company', 'Q20', q20order)

However, the data gives hope for those are new to this industry, with job-opportunities highest is in small companies (<50 employees). Besides, the opportunity to enter large companies is also optimistic.

In [ ]:
Plot_dix('Number of individuals are responsible for data science workloads', 'Q21', q21order)

But, it's probably note that, to enter the "big" company, "team work" could be an important requirement in the data science. That's nice that in the big team, we have both gender.

In [ ]:
Plot_dix('Yearly compensation distribution ($USD)', 'Q24', q24order)

And of course, income is always the most concerned question. This distribution is very interesting, it has not a clear tendency, showing several picks. And at the very high paying rate, there is NO WOMAN, are these positions very stressful?
So, what factors are related to the salary?

In [ ]:
Plot_Muldix('Pivot Table : Compensation & Contry', 'Q3', 'Q24', q24order)

The HIGH PICK salary is found in: USA, UAE, Switzeland, Israel, Canada and Austraylia. Is the data science industry very developed in these countries or is it due to differences in average income by country?

In [ ]:
Plot_Muldix('Pivot Table : Compensation & Working experience', 'Q6', 'Q24', q24order)

Wow, that's suprising, is'nt it? Even you have < 5 years of experience, you could have chance to be paid up to > $250,000 yearly. Amazing. 

In [ ]:
Plot_matrix2('Gender & Education & Salary','Q24','Q4',q24order, q4order)

And with a Master's degree, you can have a higher chance of succeeding in career development and earning an admirable salary.

... to be continued in Part 2.